In [1]:
import os
import numpy as np
from datetime import datetime
os.chdir("../")
from datasets import load_dataset, Dataset, DatasetDict
from src.data import DataProcessing, HF_Dataset

/opt/mamba/lib/python3.10/site-packages/gluonts/json.py:101: UserWarning: Using `json`-module for json-handling. Consider installing one of `orjson`, `ujson` to speed up serialization and deserialization.
  warnings.warn(


In [2]:
dataset = load_dataset("monash_tsf", "traffic_hourly")
freq = "1H"

Found cached dataset monash_tsf (/home/onyxia/.cache/huggingface/datasets/monash_tsf/traffic_hourly/1.0.0/82998723d55d6edbc664cfaebe371004d849846ee5f61b57f1f974000d44c050)


  0%|          | 0/3 [00:00<?, ?it/s]

In [3]:
data_pipeline = DataProcessing(dataset)

In [4]:
mtrain, mtest = data_pipeline.multi_variate_format(freq)

In [5]:
ds = HF_Dataset(
    [datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0), datetime(2023, 4, 12, 12, 0, 0)], 
    [1, 2, 3],
    [0.7, 0.2, 0.1],
    [0, 0, 0],
    [None, None, None],
    ["T1", "T1", "T1"]
    ).getDatasetDict()

TypeError: HF_Dataset.__init__() missing 1 required positional argument: 'item_id'

In [ ]:
ds

In [ ]:
len(dataset["train"][0]["target"])

In [ ]:
len(dataset["validation"][0]["target"])

In [11]:
np.array([0.7, 0.2, 0.1])

array([0.2, 0.1])